In [87]:
import pandas as pd
import numpy as np
import datetime as dt
import os

In [88]:
#update path to covid daily update on sharepoint site
new_data = pd.read_csv("../csse_covid_19_data/csse_covid_19_daily_reports/05-06-2020.csv")
new_data.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-05-07 02:32:28,34.223334,-82.461707,33,0,0,33,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-05-07 02:32:28,30.295065,-92.414197,140,10,0,130,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-05-07 02:32:28,37.767072,-75.632346,429,7,0,422,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-05-07 02:32:28,43.452658,-116.241552,717,19,0,698,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-05-07 02:32:28,41.330756,-94.471059,3,0,0,3,"Adair, Iowa, US"


In [89]:
sites = pd.read_excel("../clean_data/sites.xlsx")
sites.head()

,Site,Region,County,State,Combined_Key
0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US"
1,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US"


In [90]:
new_data_sites = new_data.merge(sites, left_on = "Combined_Key", right_on = "Combined_Key", how = "inner")
new_data_sites.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Site,Region,County,State
0,55061.0,Kewaunee,Wisconsin,US,2020-05-07 02:32:28,44.517275,-87.614684,25,1,0,24,"Kewaunee, Wisconsin, US",Point Beach,3,Kewaunee,Wisconsin
1,55071.0,Manitowoc,Wisconsin,US,2020-05-07 02:32:28,44.119468,-87.809289,17,1,0,16,"Manitowoc, Wisconsin, US",Point Beach,3,Manitowoc,Wisconsin


In [91]:
new_data_sites = new_data_sites[["FIPS", "Site", "Region", "County","State","Combined_Key", "Confirmed", "Last_Update"]]
new_data_sites.head()

,FIPS,Site,Region,County,State,Combined_Key,Confirmed,Last_Update
0,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",25,2020-05-07 02:32:28
1,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",17,2020-05-07 02:32:28


In [92]:
new_data_sites["Last_Update"] = pd.to_datetime(new_data_sites.Last_Update) 
new_data_sites.dtypes

FIPS                   float64
Site                    object
Region                   int64
County                  object
State                   object
Combined_Key            object
Confirmed                int64
Last_Update     datetime64[ns]
dtype: object

In [93]:
new_data_sites["Date"] = new_data_sites["Last_Update"].dt.to_period("D")

In [94]:
offset = dt.timedelta(days = 1)
new_data_sites["Date"] = new_data_sites["Date"].apply(lambda x: x-offset)
new_data_sites.drop("Last_Update", axis = 1, inplace = True)

In [95]:
new_data_sites.head()

,FIPS,Site,Region,County,State,Combined_Key,Confirmed,Date
0,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",25,2020-05-06
1,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",17,2020-05-06


In [96]:
site_time_series = pd.read_excel("../clean_data/site_time_series.xlsx", index_col = 0)

In [97]:
site_time_series.tail()

,FIPS,Site,Region,County,State,Combined_Key,Confirmed,Date
91,55071,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",13,2020-05-03
92,55061,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",22,2020-05-04
93,55071,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",14,2020-05-04
94,55061,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",22,2020-05-05
95,55071,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",16,2020-05-05


In [98]:
site_time_series = site_time_series.append(new_data_sites)
site_time_series.tail()

,FIPS,Site,Region,County,State,Combined_Key,Confirmed,Date
93,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",14,2020-05-04
94,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",22,2020-05-05
95,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",16,2020-05-05
0,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",25,2020-05-06
1,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",17,2020-05-06


In [99]:
site_time_series.reset_index(inplace = True)
site_time_series.tail()

,index,FIPS,Site,Region,County,State,Combined_Key,Confirmed,Date
93,93,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",14,2020-05-04
94,94,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",22,2020-05-05
95,95,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",16,2020-05-05
96,0,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",25,2020-05-06
97,1,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",17,2020-05-06


In [100]:
site_time_series.drop("index", axis = 1, inplace = True)
site_time_series.head()

,FIPS,Site,Region,County,State,Combined_Key,Confirmed,Date
0,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",0,2020-03-19
1,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",0,2020-03-19
2,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",0,2020-03-20
3,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",0,2020-03-20
4,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",0,2020-03-21


In [101]:
site_time_series.to_excel("../clean_data/site_time_series.xlsx")